<a href="https://colab.research.google.com/github/JayashreeNagaraj/Capstone/blob/main/Bert2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import pandas as pd
import re
import gc
import os
#print(os.listdir("../input"))
import fileinput
import string
import tensorflow as tf
import zipfile
import datetime
import sys
from tqdm  import tqdm
tqdm.pandas()
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from sklearn.metrics import classification_report
#import json as js

project_path = '/content/drive/My Drive/Colab Notebooks/CPSTN_NLP_GRP6A/'

data = pd.read_csv (r'/content/drive/My Drive/Colab Notebooks/CPSTN_NLP_GRP6A/data_modeled.csv')
print (data.head(5))
print(data.dtypes)

   Unnamed: 0  Unnamed: 0.1  ... Full Len Assignment group modified
0           0             0  ...      198                     GRP_0
1           1             1  ...      154                     GRP_0
2           2             2  ...       61                     GRP_0
3           3             3  ...       29                     GRP_0
4           4             4  ...       11                     GRP_0

[5 rows x 13 columns]
Unnamed: 0                    int64
Unnamed: 0.1                  int64
Short description            object
Description                  object
Caller                       object
Assignment group             object
slen                          int64
dlen                          int64
Full Desc                    object
language                     object
Updated Lang                 object
Full Len                      int64
Assignment group modified    object
dtype: object


In [4]:
data1 = pd.DataFrame();
data1['idx'] = data.index
data1['labels'] = data['Assignment group modified']
data1['sentence'] = data['Full Desc']
data1.dropna(how='any',inplace=True)
print(data1.head(10))

   idx labels                                           sentence
0    0  GRP_0  login issue verified user detail employee mana...
1    1  GRP_0  outlook received from hello team my meeting sk...
2    2  GRP_0  cant log in to vpn received from hi cannot log...
3    3  GRP_0                      unable to access hr_tool page
4    4  GRP_0                                        skype error
5    5  GRP_0     unable to log in to engineering tool and skype
6    6  GRP_D  event critical hostname_221 company com the va...
7    7  GRP_0  ticket_no1550391 employment status new non emp...
8    8  GRP_0                unable to disable add in on outlook
9    9  GRP_0                    ticket update on inplant_874773


In [16]:
# !pip install wget
# import wget
# url = 'https://raw.githubusercontent.com/google-research/bert/master/tokenization.py'
# filename = wget.download('https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip')
# filename
print(data1.count())

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=7c0c8de16385be4e6edb3dd4225ad7b012a3ae395341fb8cd8453d3c61ea4f16
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget
idx         8467
labels      8467
sentence    8467
dtype: int64


In [18]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/CPSTN_NLP_GRP6A/Bert')
import modeling
import optimization
import run_classifier
import tokenization

In [27]:
folder = '/content/drive/My Drive/Colab Notebooks/CPSTN_NLP_GRP6A/Bert/model_folder'
# with zipfile.ZipFile("uncased_L-12_H-768_A-12.zip","r") as zip_ref:
#     zip_ref.extractall(folder)

In [28]:
BERT_MODEL = 'uncased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/uncased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/outputs'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>>  BERT pretrained directory: {BERT_PRETRAINED_DIR}')

>> Model output directory: /content/drive/My Drive/Colab Notebooks/CPSTN_NLP_GRP6A/Bert/model_folder/outputs
>>  BERT pretrained directory: /content/drive/My Drive/Colab Notebooks/CPSTN_NLP_GRP6A/Bert/model_folder/uncased_L-12_H-768_A-12


In [29]:
data1['labels'] = LabelEncoder().fit_transform(data1['labels'])
data1['labels'].unique()

array([0, 5, 3, 4, 1, 2])

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data1["sentence"].values, data1["labels"].values, test_size=0.2, random_state=42)

In [31]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

In [32]:
# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 3.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 50
# Model configs
SAVE_CHECKPOINTS_STEPS = 100000
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

In [33]:
label_list = [str(num) for num in range(6)]
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.compat.v1.estimator.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.compat.v1.estimator.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.compat.v1.estimator.tpu.InputPipelineConfig.PER_HOST_V2))

In [34]:
num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.compat.v1.estimator.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/Colab Notebooks/CPSTN_NLP_GRP6A/Bert/model_folder/outputs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=100000, num_shards=8, num_cores_per_repli

In [35]:
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.compat.v1.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

Please wait...
INFO:tensorflow:Writing example 0 of 6773
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train
INFO:tensorflow:tokens: [CLS] user want to log in to info ##net [SEP]
INFO:tensorflow:input_ids: 101 5310 2215 2000 8833 1999 2000 18558 7159 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train
INFO:tensorflow:tokens: [CLS] job sid _ 37 ##hot ##i failed in job _ schedule ##r at received from job sid _ 37 ##hot ##i failed in job _ schedule ##r at [SEP]
INFO:tensorflow:input_ids: 101 3105 15765 1035 4261 12326 2072 3478 1999 3105 1035 6134 2099 2012 2363 2013 3105 15765 1035 4261 12326 2072 3478 1

In [36]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 500

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [37]:
predict_examples = create_examples(X_test, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

INFO:tensorflow:Writing example 0 of 1694
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test
INFO:tensorflow:tokens: [CLS] company ##gues ##t access name er ##ir ##tc language browser microsoft internet explorer email da ##bh ##ru ##ji customer number telephone summary hi er ##ir ##tc wi ##fi access for the following is not working dt ##by ##cs ##gf v ##f ##d ##gl ##q ##np sa ##z ##to [SEP]
INFO:tensorflow:input_ids: 101 2194 22967 2102 3229 2171 9413 4313 13535 2653 16602 7513 4274 10566 10373 4830 23706 6820 4478 8013 2193 7026 12654 7632 9413 4313 13535 15536 8873 3229 2005 1996 2206 2003 2025 2551 26718 3762 6169 25708 1058 2546 2094 23296 4160 16275 7842 2480 3406 102
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:***

In [77]:
import numpy as np
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

In [78]:
#preds.append(np.argmax(prediction['probabilities']))
preds1=preds
preds1.append(0)

In [79]:
from sklearn.metrics import accuracy_score
print("Accuracy of BERT is:",accuracy_score(y_test,preds1))

ValueError: ignored

In [80]:
print(preds)

[0]


In [76]:
print(classification_report(y_test,preds1))

              precision    recall  f1-score   support

           0       0.43      0.03      0.06       802
           1       0.03      0.05      0.04        39
           2       0.04      0.20      0.06        59
           3       0.05      0.04      0.04       131
           4       0.03      0.19      0.06        62
           5       0.33      0.44      0.38       601

    accuracy                           0.19      1694
   macro avg       0.15      0.16      0.11      1694
weighted avg       0.33      0.19      0.17      1694

